# This is just a simplified version of my edge picking notebook, so you can clearly see how we chose which mutations to use in the second experiment (TP)

In [1]:
import pandas as pd
import numpy as np

# Loading in data from Sergey's analysis and annotation data

In [2]:
sd = pd.read_csv('sk_data_headers_changed.csv')
sd['new.num.sig'] = sd.apply(lambda row: len([i for i in [j for j in range(1, 21) if j not in [9, 18]] if row['sig.Seg' + str(i).zfill(2)]]), axis=1)
ann_dat = pd.read_csv('../../Mutation_Annotation/Edges_annotated.csv')
mg = sd.merge(ann_dat, on='Edge', how='left')

In [3]:
def clearly_neutral(row):
    neut = 0
    for i in [j for j in range(1,21) if j not in [9,18]]:
        s = row['s.Seg' + str(i).zfill(2)] 
        if not np.isnan(s) and np.abs(s) < 0.5:
            neut += 1
    return neut

putative_neutral = mg.loc[mg.apply(lambda x: clearly_neutral(x) == 18, axis=1)].loc[mg.apply(lambda x: str(x['Gene'])=='nan', axis=1)].loc[mg['Expected.From.TnCS']]
print(len(putative_neutral))

30


## From the above list of neutral edges I chose 5 that looked good by eye:

In [4]:
chosen_neutral_list = {
    418: 'CGATGATGATGATGAAACGAAAAACTGCTA',
    691: 'GTTGGGCGCTTAAAGAAGAGACTCACTAAA',
    742: 'TATATTGAACTTTACAACTTGTACCCTTCA',
    757: 'TCAAAACGGAGTGTTTACAGCTATCTGTCC',
    73: 'ACAACCTACCTGCTATCTTGTTCACGGATA' 
    }
neutral_choice = mg.loc[mg['ID'].isin(chosen_neutral_list)]

# Assembling the chosen set:
* the neutral edges chosen above
* all edges that have significant fitness effects in at least 6 clones, except one (the third FUN30 one, since it is the same as the other two (and between them on the gene))

In [5]:
mg.loc[mg['new.num.sig']>5].loc[mg['ORF'] == 'YAL019W']

,ID,Edge,s.Seg01,s.Seg02,s.Seg03,s.Seg04,s.Seg05,s.Seg06,s.Seg07,s.Seg08,...,orf_strand.nearby,phenotypeSummary,phenotypeSummary.nearby,start,start.nearby,Expected.From.TnCS,In.TP,TP.Control,Expected.In.TP,TP.Reference
4,760,TCAAATGATGAGGATGACGTAGTGCAAGTG,-0.33,-2.04,-2.89,-3.03,-2.57,-2.15,-3.54,0.28,...,1|-1|1,"Non-essential gene, null mutant shows decrease...",None|Non-essential gene; null mutant displays ...,114919,114250|113614|114919,True,True,False,True,False
16,79,ACAAGTTTTACAGTTTATTAATGACTCTGA,-0.28,-1.35,-2.73,-3.93,-2.86,-3.30,-4.46,0.50,...,1,"Non-essential gene, null mutant shows decrease...","Non-essential gene, null mutant shows decrease...",114919,114919,True,True,False,True,False
91,278,ATTGGCAAGTCTTATGAGTCATCCGCTACA,-0.47,-0.97,-2.93,-2.35,-3.03,-2.00,-3.21,0.40,...,1,"Non-essential gene, null mutant shows decrease...","Non-essential gene, null mutant shows decrease...",114919,114919,True,False,False,False,False


In [8]:
sig_choice = mg.loc[mg['new.num.sig']>5].loc[mg['Expected.From.TnCS']].loc[mg['Edge'] != 'ATTGGCAAGTCTTATGAGTCATCCGCTACA']
sig_choice['Is.Reference'] = [False for i in range(len(sig_choice))]
neutral_choice['Is.Reference'] = [True for i in range(len(neutral_choice))]
full_choice = pd.concat([sig_choice, neutral_choice])

/n/home13/mjohnson/.conda/envs/milo_simple_conda5/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Confirming that this set is the same one I chose before:

In [9]:
d = pd.read_csv('Tn96_edges_chosen_final.csv')
print(len(d), len(full_choice), len(set(d['Edge']).intersection(set(full_choice['Edge']))))

96 96 96


## great